In [19]:
#import libraries
import boto3
import csv
import json
import time

In [3]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
prefix = "english_documents/raw_pdf/"
i = 0
for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
    srcKey = object.key
    if ".txt" in srcKey:
        print(i)
        newKey = srcKey.replace(".txt", ".pdf")
        copySource = s3BucketName + '/' + srcKey         
        s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
        s3.Object(s3BucketName, srcKey).delete()
    i += 1

3
5
6
8
9
11
12
13
14
15
17
20
22
23
25
26
27
28
29
30
31
38
39
41
42
43
44
45
46
47
48
49
50
54
57
58
59
61
62
63
69
73
75
76
77
80
81
82
83
84
85
86
87
88
89
95
97
98
101
103
104
106
107
108
110
111
112
113
114
115
116
118
119
120
123
124
126
128
131
135
136
138
139
140
142
143
144
146
150
151
155
163
165
166
168
170
171
174
175
180
181
184
185
186
188
189
191
192
194
195
197
198
199
201
202
203
204
205
206
209
210
213
217
221
222
224
228
229
231
236
237
239
240
243
244
245
246
247
250
251
252
256
257
260
266
267
268
269
270
271
273
274
276
279
280
281
283
284
285
286
291
292
294
300
301
302
304
306
307
309
312
313
315
316
319
320
322
323
324
325
326
327
328
331
332
333
335
336
337
339
340
341
342
345
346
347
352
354
356
358
359
361
363
368
369
370
373
374
376
377
378
379
380
382
386
387
390
393
399
400
404
405
406
407
408
409
410
411
412
413
418
419
420
421
422
425
426
427
429
431
432
434
435
438
439
440
442
443
444
449
453
455
456
457
458
460
464
469
470
472
473
474
475
476
477
480

In [14]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [28]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [ ]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "english_documents/raw_pdf/"
i = 0


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key and i > 48: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        key = obj.key.replace("raw_pdf", "text_files").replace(".pdf", ".txt")
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

49 ** english_documents/raw_pdf/12188400da2085e7ae71987a5e0957d49eb50ee3.pdf
50 ** english_documents/raw_pdf/121df7be99f04def25bc292e099d8a13363b5597.pdf
51 ** english_documents/raw_pdf/124e3b453992b411b8aab4e35fbd815645a4e4db.pdf
52 ** english_documents/raw_pdf/125e0cd5c793f4d05b7d6f7aafad189e9509d3ed.pdf
53 ** english_documents/raw_pdf/12c777767328bb2c20dee852ca68b7642e243be0.pdf
54 ** english_documents/raw_pdf/12f88f758451d41f940412b10fe34d2dc1e43349.pdf
55 ** english_documents/raw_pdf/13102d4c1abf5ebbd286b644e9a597042b5ce0aa.pdf
56 ** english_documents/raw_pdf/13576db1d6da3cec7905b603bfd3dca5c9fb8226.pdf
57 ** english_documents/raw_pdf/135b40560085c86ac8d0c5775ae3e312d5a88d90.pdf
58 ** english_documents/raw_pdf/139fd4dae0876c1a13984db6a4c16597a3d35966.pdf
59 ** english_documents/raw_pdf/13cd93fb4eb6d119db7da689915e46c5bb9eafe8.pdf
60 ** english_documents/raw_pdf/144a86ffefe02f757b276a6e6f3ab4e971781a06.pdf
61 ** english_documents/raw_pdf/15b7ef405a5efbaaa17731f3f1eea30f35096214.pdf

In [ ]:
policy